In [ ]:
# Importing required python modules

import os
import cv2
import csv
import math
import h5py
import glob
import scipy
import shutil
import random
import argparse

import pandas as pd
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.io import loadmat
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image, ImageOps
from imageio import imwrite, imread
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
%matplotlib inline

In [ ]:
# Setting up working directory

dataset_part = 'part_B'
working_input_folder = '../working/input/shanghaitech/'

# making the required folders in working directory
for i in [working_input_folder]:
    if not os.path.exists(i):
        os.makedirs(i)

In [ ]:
# Copy input directory dataset into working directory

dataset_initial_path = '../input/shanghaitech/ShanghaiTech/'
dataset_copy_path = '../working/input/shanghaitech/ShanghaiTech/'

# copying the dataset directory tree into working directory
shutil.copytree(dataset_initial_path, dataset_copy_path)
print('copying completed')

In [ ]:
# Percentage of low light and night time images
method_1_night_images_percent = 0.3
method_2_night_images_percent = 0.4

# Number of images or outputs to be displayed
number_of_degraded_images_to_display = 8
number_of_enhanced_images_to_display = 8

In [ ]:
# Function to display original image and corresponding degraded image
def display_degraded_image(image_path, original_image, degraded_image):
    print(image_path)
    
    # initializing a subplot of size 1*2
    f, axarr = plt.subplots(1, 2, figsize=(10,10))

    # assigning images to subplot
    axarr[0].imshow(original_image)
    axarr[1].imshow(degraded_image)
    
    # removing ticks from both axes
    axarr[0].set_xticks([])
    axarr[0].set_yticks([])
    axarr[1].set_xticks([])
    axarr[1].set_yticks([])
    
    # set plot titles
    axarr[0].title.set_text('original image')
    axarr[1].title.set_text('degraded image')

In [ ]:
# Converts test data to low light and night time images

# Testing images data path
test_data_path = dataset_copy_path + dataset_part + '/test_data/images/'

# testing images loaded into memory
image_files = [filename for filename in os.listdir(test_data_path)]
num_images = len(image_files)
indices = list(range(1, num_images + 1))

# number of images for both types of degradation
method_1_images = int(num_images * method_1_night_images_percent)
method_2_images = int(num_images * method_2_night_images_percent)

# degradation of images by method-1 to night time images
for idx in range(method_1_images):
    ind = indices[idx]
    
    # printing number of images degraded so far
    if (idx+1) % 10 == 0:
        print('Processing {}/{} files'.format(idx+1, num_images))
    
    # loading images for degradation
    input_img_name = ''.join((test_data_path, 'IMG_', str(ind), '.jpg'))    
    img = imread(input_img_name)

    # generating and saving night time images
    arr = img * np.array([0.2,0.4,0.7])
    arr2 = (255*arr/arr.max()).astype(np.uint8)
    imwrite(input_img_name, arr2)
    
    # printing degraded night time images
    if idx < number_of_degraded_images_to_display/2:
        display_degraded_image(input_img_name, img, arr2)
    
    
# degradation of images by method-2 to low light images
for idx in range(method_2_images):
    ind = indices[idx+method_1_images]
    
    # printing number of images degraded so far
    if (idx+1) % 10 == 0:
        print('Processing {}/{} files'.format(idx+1, num_images))
    
    # loading images for degradation
    input_img_name = ''.join((test_data_path, 'IMG_', str(ind), '.jpg'))
    img = load_img(input_img_name)
    
    # convert image to numpy array representation
    data = img_to_array(img)
    
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    
    # create image data augmentation generator for degrading brightness level of image
    datagen = ImageDataGenerator(brightness_range=[0.2,0.7])
    
    # prepare iterator, generate and save low light image
    it = datagen.flow(samples, batch_size=1)
    batch = it.next()
    image = batch[0].astype('uint8')
    imwrite(input_img_name, image)

    # printing degraded low light images
    if idx < number_of_degraded_images_to_display/2:
        display_degraded_image(input_img_name, img, image)

In [ ]:
# number of patches to be taken from each image
num_of_patches = 16

# number of images in training dataset
train_dataset_size = 400

# setting paths to dataset directories 
dataset_base_path = dataset_copy_path + dataset_part + '/'
output_base_path = '../working/shanghaitech/' + dataset_part + '/'
train_images_path = dataset_base_path + 'train_data/images/'
train_gt_path = dataset_base_path + 'train_data/ground-truth/'
train_gt_den_map_csv_path = output_base_path + 'train_data/ground_truth_den_map_csv/'

# setting path to store patches data
patches_dataset_path = output_base_path + 'patches_data/'
patches_path_train_images = patches_dataset_path + 'train_images/'
patches_path_train_den_map = patches_dataset_path + 'train_den_map/'
patches_path_val_images = patches_dataset_path + 'val_images/'
patches_path_val_den_map = patches_dataset_path + 'val_den_map/'

# creating required directories
for i in [train_gt_den_map_csv_path, patches_dataset_path, patches_path_train_images, patches_path_train_den_map, patches_path_val_images, patches_path_val_den_map]:
    if not os.path.exists(i):
        os.makedirs(i)

In [ ]:
# Function to generate density map from given annotated people heads using gaussian kernel

def gen_density_map(img, anno_points):
    # initializing density map with all zeroes
    density_map = np.zeros_like(img, dtype=np.float64)
    h, w = density_map.shape
    
    # Gaussian kernel size
    kernel_size = 15 
    # standard deviation
    sigma = 4.0 

    for point in anno_points:
        # Center point coordinates of human head
        x, y = min(w-1, abs(math.floor(point[0]))), min(h-1, abs(math.floor(point[1])))

        # Upper left corner coordinates and lower right corner coordinates
        x1, y1 = x-kernel_size // 2, y-kernel_size // 2
        x2, y2 = x + kernel_size // 2 + 1, y + kernel_size // 2 + 1

        # Out of bounds offset
        dx1, dy1, dx2, dy2 = 0, 0, 0, 0 
        out_of_bounds = False
        
        # Following four ifs are used to determine whether the x and y of the two top corners are out of bounds
        if x1 <0:
            dx1 = abs(x1)
            x1 = 0
            out_of_bounds = True
        if y1 <0:
            dy1 = abs(y1)
            y1 = 0
            out_of_bounds = True
        if x2> w:
            dx2 = x2-w
            x2 = w
            out_of_bounds = True
        if y2> h:
            dy2 = y2-h
            y2 = h
            out_of_bounds = True

        # If it is out of bounds, adjust the size of the Gaussian kernel
        if out_of_bounds:
            kernel_h = kernel_size-dy1-dy2
            kernel_w = kernel_size-dx1-dx2

            # Generate a Gaussian kernel of size (kernel_h, kernel_w)
            H = np.multiply(cv2.getGaussianKernel(kernel_h, sigma), (cv2.getGaussianKernel(kernel_w, sigma)).T)
        else:
            # Generate a Gaussian kernel of size (15, 15)
            H = np.multiply(cv2.getGaussianKernel(kernel_size, sigma), (cv2.getGaussianKernel(kernel_size, sigma)).T)

        density_map[y1:y2, x1:x2] += H
        
    return density_map

In [ ]:
# Displays count of the number of people in each image in training dataset

num_images = train_dataset_size
people_count_train_images = []

for idx in range(num_images):
    i = idx + 1

    mat_img = loadmat(''.join((train_gt_path, 'GT_IMG_', str(i), '.mat')))
    image_info = mat_img['image_info']
    annPoints = image_info[0][0][0][0][0]
    people_count_train_images.append(len(annPoints))

print(people_count_train_images)

In [ ]:
# Number of images in which count of people is less than 200
cnt_less_than_equal_to_200 = 0
for i in people_count_train_images:
    if i <= 200:
        cnt_less_than_equal_to_200 += 1
print(cnt_less_than_equal_to_200)

In [ ]:
# filter out images in which count of people less than 200
count_in_train_images = [count for count in people_count_train_images if count <= 200]
print(len(count_in_train_images))

In [ ]:
# visualizes the training dataset images

fig,ax = plt.subplots(figsize=(5,5))
ax.plot(range(len(count_in_train_images)),count_in_train_images)
ax.legend()
ax.title.set_text('Number of people in Part B training images')
ax.set_xlabel('number of images')
ax.set_ylabel('number of people')

In [ ]:
# This code saves the density map of training images as csv files

num_images = train_dataset_size
# number of images for validation purpose
num_val = math.ceil(num_images * 0.1)  

for idx in range(num_images):
    # remove images in which people count is greater than 200
    if people_count_train_images[idx] > 200:
        continue
        
    i = idx + 1
    if i % 10 == 0:
        print('Processing {}/{} files'.format(i, num_images))
    
    input_img_name = ''.join((train_images_path, 'IMG_', str(i), '.jpg'))
    
    # loading image and ground truth file
    if os.path.isfile(input_img_name):
        im = cv2.imread(input_img_name, 0)
        mat_img = loadmat(''.join((train_gt_path, 'GT_IMG_', str(i), '.mat')))
        image_info = mat_img['image_info']
        # annotated head pixel coordinates
        annPoints = image_info[0][0][0][0][0]
        
        # generate density map
        im_density = gen_density_map(im, annPoints)
        
        # setting density map name and path and saving it
        with open(''.join([train_gt_den_map_csv_path, str(i), '.csv']), 'w', newline='') as fout:
            writer = csv.writer(fout)
            writer.writerows(im_density)

In [ ]:
# setting number of images to display in output

num_of_train_images_to_display = 5
num_of_test_images_to_display = 10

In [ ]:
# loading and shuffling training images
train_image_files = [filename for filename in os.listdir(train_gt_den_map_csv_path)]
random.shuffle(train_image_files)

print(len(train_image_files))

In [ ]:
# this code displays the generated density map for training images from annotated heads data in ground truth files

for idx in range(num_of_train_images_to_display):
    i = int(train_image_files[idx].split('.')[0])
    
    # initializing a subplot of 1*2
    f, axarr = plt.subplots(1, 2, figsize=(10,10))
    
    input_img_name = ''.join((train_images_path, 'IMG_', str(i), '.jpg'))
    
    # load and display image and density map
    if os.path.isfile(input_img_name):
        axarr[0].imshow(Image.open(input_img_name))

        im_density = np.loadtxt(open(''.join([train_gt_den_map_csv_path, str(i), '.csv']), "rb"), delimiter=",")
        axarr[1].imshow(im_density, interpolation='nearest')

        # removing ticks from both axes
        axarr[0].set_xticks([])
        axarr[0].set_yticks([])
        axarr[1].set_xticks([])
        axarr[1].set_yticks([])
        
        # setting plot x and y labels
        axarr[0].set_xlabel('original image')
        axarr[1].set_xlabel('density map')

In [ ]:
# This code saves the density map and cropped images of the patches of training and validation data

num_images = train_dataset_size
# number of images for validation purpose
num_val = math.ceil(num_images * 0.1)

# training and validation images count
new_train_images_cnt = 0
new_val_images_cnt = 0

for idx in range(num_images):
    # remove images in which peopl count is greater than 200
    if people_count_train_images[idx] > 200:
        continue
        
    i = idx + 1
    if i % 10 == 0:
        print('Processing {}/{} files'.format(i, num_images))
    
    input_img_name = ''.join((train_images_path, 'IMG_', str(i), '.jpg'))

    if os.path.isfile(input_img_name):
        # load image
        im = cv2.imread(input_img_name, 0)
        
        # load ground truth file
        mat_img = loadmat(''.join((train_gt_path, 'GT_IMG_', str(i), '.mat')))
        image_info = mat_img['image_info']
        annPoints = image_info[0][0][0][0][0]
      
        im_density = gen_density_map(im, annPoints)
        
        # counting training and validation images
        if (idx+1) < (num_images-num_val):
            new_train_images_cnt += 1
        else:
            new_val_images_cnt += 1
    
        h, w = im.shape
        # setting patch size to be (h/4, w/4)
        patch_h, patch_w = int(h/4), int(w/4)

        # extract patches data from image
        j = 0
        for l in range(4):
            for k in range(4):
                # calculating corner points of the patch to be cropped
                x1 = k*patch_w
                y1 = l*patch_h
                x2 = (k+1)*patch_w
                y2 = (l+1)*patch_h
                j += 1
                
                # crop image and the density map for the patch
                im_sampled = im[y1:y2, x1:x2]
                im_density_sampled = im_density[y1:y2, x1:x2]

                # setting patch unique name and path for image and density map and saving both of them
                img_idx = ''.join((str(i), '_', str(j)))
                path_img, path_den = (patches_path_train_images, patches_path_train_den_map) if (idx+1) < (num_images-num_val) else (patches_path_val_images, patches_path_val_den_map)
                cv2.imwrite(''.join([path_img, img_idx, '.jpg']), im_sampled)
                
                with open(''.join([path_den, img_idx, '.csv']), 'w', newline='') as fout:
                    writer = csv.writer(fout)
                    writer.writerows(im_density_sampled)

In [ ]:
# Setting patches path for input data to MCNN model

train_path = patches_path_train_images
train_den_path = patches_path_train_den_map
val_path = patches_path_val_images
val_den_path = patches_path_val_den_map

In [ ]:
# This class loads the training and validation data of the patches

class DataLoader(object):
    def __init__(self, data_path, gt_path, shuffle=False, gt_downsample=False):
         
        # initialize path to input data
        self.data_path = data_path
        self.gt_path = gt_path
        self.shuffle = shuffle
        self.gt_downsample = gt_downsample
        
        # load files
        self.data_files = [filename for filename in os.listdir(gt_path)]
        self.num_samples = len(self.data_files)
        self.blob_list = []

        for fname in self.data_files:
            # load image as grayscale
            img = cv2.imread(os.path.join(self.data_path, os.path.splitext(fname)[0] +'.jpg'), 0)
            img = img.astype(np.float32, copy=False)
            ht = img.shape[0]
            wd = img.shape[1]
            
            ht_1 = int((ht / 4) * 4)
            wd_1 = int((wd / 4) * 4)
            
            # resizing and reshaping image to one dimension
            img = cv2.resize(img, (wd_1, ht_1))
            img = img.reshape((img.shape[0], img.shape[1], 1))
            
            # reading density map
            den = pd.read_csv(os.path.join(self.gt_path, fname),
                              header=None).values
            den = den.astype(np.float32, copy=False)
            if self.gt_downsample:
                wd_1 = int(wd_1 / 4)
                ht_1 = int(ht_1 / 4)
            den = cv2.resize(den, (wd_1, ht_1))
            den = den * ((wd * ht) / (wd_1 * ht_1))
            den = den.reshape((den.shape[0], den.shape[1], 1))
            
            # creating object for patch data
            blob = dict()
            blob['data'] = img
            blob['gt'] = den
            blob['fname'] = fname
            self.blob_list.append(blob)

        if self.shuffle:
            np.random.shuffle(self.blob_list)

    # returning data in batches
    def flow(self, batch_size=32):
        loop_count = self.num_samples // batch_size
        while True:
            np.random.shuffle(self.blob_list)
            for i in range(loop_count):
                blobs = self.blob_list[i*batch_size: (i+1)*batch_size]
                X_batch = np.array([blob['data'] for blob in blobs])
                Y_batch = np.array([blob['gt'] for blob in blobs])
                yield X_batch, Y_batch
    
    # returring whole data
    def get_all(self):
        X = np.array([blob['data'] for blob in self.blob_list])
        Y = np.array([blob['gt'] for blob in self.blob_list])
        return X, Y
    
    # iterate over data
    def __iter__(self):
        for blob in self.blob_list:
            yield blob

In [ ]:
# loads patches data for training and validation using DataLoader class

print('Loading data, wait a moment...')
train_data_gen = DataLoader(train_path, train_den_path, shuffle=True, gt_downsample=True)
val_data_gen = DataLoader(val_path, val_den_path, shuffle=False, gt_downsample=True)

print('done')
print(train_data_gen.num_samples)
print(val_data_gen.num_samples)

In [ ]:
# code for visualization of patches data

ct_gts = []
xx=0
for blob in train_data_gen:
    xx=xx+1
    gt = blob['gt']
    gt_count = np.sum(gt)
    ct_gts.append(gt_count)

print(len(ct_gts))

# plot to show number of people in each training patch
fig,ax = plt.subplots(figsize=(5,5))
ax.plot(range(train_data_gen.num_samples),ct_gts,label='training')
ax.legend()
ax.set_xlabel('number of images')
ax.set_ylabel('number of people')

ct_gts2 = []
xx=0
for blob2 in val_data_gen:
    xx=xx+1
    gt2 = blob2['gt']
    gt_count2 = np.sum(gt2)
    ct_gts2.append(gt_count2)

print(len(ct_gts2))

# plot to show number of people in each validation patch
fig2,ax2 = plt.subplots(figsize=(5,5))
ax2.plot(range(val_data_gen.num_samples),ct_gts2,label='validation')
ax2.legend()
ax2.set_xlabel('number of images')
ax2.set_ylabel('number of people')

In [ ]:
# defining regression evaluation metrics

def mae(y_true, y_pred):
    return K.abs(K.sum(y_true) - K.sum(y_pred))

def mse(y_true, y_pred):
    return (K.sum(y_true) - K.sum(y_pred)) * (K.sum(y_true) - K.sum(y_pred))

In [ ]:
# Making the MCNN model

# model uses convolution layer and max pooling layer with relu as activation function
def MCNN(input_shape=None):
    inputs = Input(shape=input_shape)

    # column 1
    column_1 = Conv2D(16, (9, 9), padding='same', activation='relu')(inputs)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = (column_1)
    column_1 = Conv2D(32, (7, 7), padding='same', activation='relu')(column_1)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = Conv2D(16, (7, 7), padding='same', activation='relu')(column_1)
    column_1 = Conv2D(8, (7, 7), padding='same', activation='relu')(column_1)

    # column 2
    column_2 = Conv2D(20, (7, 7), padding='same', activation='relu')(inputs)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = (column_2)
    column_2 = Conv2D(40, (5, 5), padding='same', activation='relu')(column_2)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = Conv2D(20, (5, 5), padding='same', activation='relu')(column_2)
    column_2 = Conv2D(10, (5, 5), padding='same', activation='relu')(column_2)

    # column 3
    column_3 = Conv2D(24, (5, 5), padding='same', activation='relu')(inputs)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = (column_3)
    column_3 = Conv2D(48, (3, 3), padding='same', activation='relu')(column_3)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = Conv2D(24, (3, 3), padding='same', activation='relu')(column_3)
    column_3 = Conv2D(12, (3, 3), padding='same', activation='relu')(column_3)

    # merge feature map of 3 columns in last dimension
    merges = Concatenate(axis=-1)([column_1, column_2, column_3])
    
    # density map generation
    density_map = Conv2D(1, (1, 1), padding='same')(merges)

    model = Model(inputs=inputs, outputs=density_map)
    return model

In [ ]:
# setting path to save trained model

trained_model_path = '../working/shanghaitech/trained_models/' + dataset_part + '/'

# creating directory to store trained model
for i in [trained_model_path]:
    if not os.path.exists(i):
        os.makedirs(i)

In [ ]:
# Initializing the MCNN model

input_shape = (None, None, 1)
model = MCNN(input_shape)

# setting model optimizers and loss parameters
adam = Adam(learning_rate=1e-6)
model.compile(loss='mse', optimizer=adam, metrics=[mae, mse])

model_name = 'model1.h5'

# using model checkpointing technique to save model whenever the model accuracy has improved
checkpointer_best_train = ModelCheckpoint(
    filepath=trained_model_path + model_name,
    monitor='loss', verbose=1, save_best_only=True, mode='min'
)

callback_list = [checkpointer_best_train]

print('done prepare {} ...'.format(dataset_part))
model.summary()

In [ ]:
# training the MCNN model 

print ("start train..........")
h = model.fit(train_data_gen.flow(1),
                    steps_per_epoch=train_data_gen.num_samples // 1,
                    validation_data=val_data_gen.flow(1),
                    validation_steps=val_data_gen.num_samples // 1,
                    batch_size=10,
                    epochs=100,
                    callbacks=callback_list,
                    verbose=1)

In [ ]:
# loading the trained model

import tensorflow as tf

trained_model_path = '../working/shanghaitech/trained_models/' + dataset_part + '/' + model_name
model = tf.keras.models.load_model(trained_model_path)

In [ ]:
# setting path to test images directory

test_images_path = dataset_base_path + 'test_data/images/'
test_gt_path = dataset_base_path + 'test_data/ground-truth/'

In [ ]:
# Outputs the count of number of people in each image in test dataset

# test images sample size
test_dataset_size = 316

num_images = test_dataset_size
people_count_test_images = []

for idx in range(num_images):
    i = idx + 1
    
    # load ground truth file
    mat_img = loadmat(''.join((test_gt_path, 'GT_IMG_', str(i), '.mat')))
    image_info = mat_img['image_info']
    annPoints = image_info[0][0][0][0][0]
    people_count_test_images.append(len(annPoints))

print(people_count_test_images)

In [ ]:
#  count test images that have people count less than equal to 200
cnt_less_than_equal_to_200_test = 0
for i in people_count_test_images:
    if i <= 200:
        cnt_less_than_equal_to_200_test += 1
print(cnt_less_than_equal_to_200_test)

In [ ]:
# filtering out test images that has count less than equal to 200
count_in_test_images = [count for count in people_count_test_images if count <= 200]
print(len(count_in_test_images))

In [ ]:
# this code visualizes the count of people in test dataset

fig,ax = plt.subplots(figsize=(5,5))
ax.plot(range(len(count_in_test_images)),count_in_test_images)
ax.legend()
ax.title.set_text('Number of people in test images')
ax.set_xlabel('number of images')
ax.set_ylabel('number of people')

In [ ]:
# setting path to results directory

results_dir = '../working/shanghaitech/results/' + dataset_part + '/'
heatmaps_dir = results_dir + 'den_map/'
txt_dir = results_dir + 'text/'
test_gt_den_map_csv_path = output_base_path + 'test_data/ground_truth_den_map_csv/'

# creating result directories
for i in [results_dir, heatmaps_dir, txt_dir, test_gt_den_map_csv_path]:
    if not os.path.exists(i):
        os.makedirs(i)

In [ ]:
# This code saves the density map of testing images as csv files

num_images = test_dataset_size

for idx in range(num_images):
    # remove images in which people count is greater than 200
    if people_count_test_images[idx] > 200:
        continue
        
    i = idx + 1
    if i % 10 == 0:
        print('Processing {}/{} files'.format(i, num_images))
    
    input_img_name = ''.join((test_images_path, 'IMG_', str(i), '.jpg'))
    
    if os.path.isfile(input_img_name):
        # load image
        im = cv2.imread(input_img_name, 0)

        mat_img = loadmat(''.join((test_gt_path, 'GT_IMG_', str(i), '.mat')))
        image_info = mat_img['image_info']
        annPoints = image_info[0][0][0][0][0]

        # generate density map
        im_density = gen_density_map(im, annPoints)

        # setting density map name and path and saving it
        with open(''.join([test_gt_den_map_csv_path, 'IMG_', str(i), '.csv']), 'w', newline='') as fout:
            writer = csv.writer(fout)
            writer.writerows(im_density)

In [ ]:
# load testing sample
print('Loading data, wait a moment...')
test_data_loader = DataLoader(test_images_path, test_gt_den_map_csv_path, shuffle=False, gt_downsample=True)
print(test_data_loader.num_samples)

In [ ]:
# Prediction of people in images using trained model

print('Testing {} ...'.format(dataset_part))

# variable to store mean absolute error and mean squared error
mae = 0.0
mse = 0.0
u=0

# dictionary to store predictions
predictions = {}
ct_preds = []
# list to store actual values
ct_gts = []
xx=0
for blob in test_data_loader:
    xx=xx+1
    img = blob['data']
    gt = blob['gt']
    
    # predicting density map and people count from trained model for testing images
    pred = model.predict(np.expand_dims(img, axis=0))
    gt_count = np.sum(gt)
    pred_count = np.sum(pred)
    ct_preds.append(pred_count)
    ct_gts.append(gt_count)
    
    image_no = blob['fname'].split('.')[0][4:]
    predictions[image_no] = pred_count
  
    # calculating absolute error and squared error 
    mae += abs(gt_count - pred_count)
    mse += ((gt_count - pred_count) * (gt_count - pred_count))
    
    # create and save heatmap
    pred = np.squeeze(pred)  # shape(1, h, w, 1) -> shape(h, w)
    
    #save_heatmap
    with open(''.join([heatmaps_dir, 'IMG_', str(image_no), '.csv']), 'w', newline='') as fout:
          writer = csv.writer(fout)
          writer.writerows(pred)
        
    # save results
    print('<{}> {:.2f} -- {:.2f}\n'.format(blob['fname'].split('.')[0], gt_count, pred_count))
    with open(txt_dir + 'predictions.txt', 'a') as f:
        line = '<{}> {:.2f} -- {:.2f}\n'.format(blob['fname'].split('.')[0], gt_count, pred_count)
        f.write(line)

In [ ]:
# load and shuffle test images
test_image_files = [filename for filename in os.listdir(test_gt_den_map_csv_path)]
random.shuffle(test_image_files)
print(len(test_image_files))

In [ ]:
# displays the original image, ground truth and predicted density map for test images

for idx in range(num_of_test_images_to_display):
    i = int(test_image_files[idx].split('.')[0][4:])
    
    # initializing a subplot of size 1*3
    f, axarr = plt.subplots(1, 3, figsize=(10,10))
    
    input_img_name = ''.join((test_images_path, 'IMG_', str(i), '.jpg'))
    
    if os.path.isfile(input_img_name):
        axarr[0].imshow(Image.open(input_img_name))

        im_density = np.loadtxt(open(''.join([test_gt_den_map_csv_path, 'IMG_', str(i), '.csv']), "rb"), delimiter=",")
        axarr[1].imshow(im_density, interpolation='nearest')

        im_density = np.loadtxt(open(''.join([heatmaps_dir, 'IMG_', str(i), '.csv']), "rb"), delimiter=",")
        axarr[2].imshow(im_density, interpolation='nearest')

        # removing ticks from both axes
        axarr[0].set_xticks([])
        axarr[0].set_yticks([])
        axarr[1].set_xticks([])
        axarr[1].set_yticks([])
        axarr[2].set_xticks([])
        axarr[2].set_yticks([])
        
        # setting x-label
        axarr[0].set_xlabel('original image')
        axarr[1].set_xlabel('ground truth')
        axarr[2].set_xlabel('predicted density map')
        
        # setting plot title
        axarr[1].title.set_text('original count: ' + str(people_count_test_images[i-1]))
        axarr[2].title.set_text('predicted count: ' + str(int(predictions[str(i)])))

In [ ]:
# calculating mean absolute error and mean squared error
avg_mae = mae / test_data_loader.num_samples
avg_mse = np.sqrt(mse / test_data_loader.num_samples)
print("mae - ", avg_mae)
print("mse - ", avg_mse)

In [ ]:
# visualizing model predictions with actual values
plt.plot(ct_preds, 'r>')
plt.plot(ct_gts, 'b+')
plt.legend(['prediction', 'ground truth'])
plt.xlabel('number of image')
plt.ylabel('number of people')
plt.title('Prediction vs Ground Truth')
plt.show()

error = np.array(ct_preds) - np.array(ct_gts)
plt.plot(error)
plt.xlabel('number of image')
plt.ylabel('difference in count of people')
plt.title('Prediction Count - Ground Truth Count')
plt.show()

idx_max_error = np.argsort(np.abs(error))[::-1]
print('MEAN = {}, MAE = {}, MSE = {}'.format(
    str(round(np.mean(error), 3)),
    str(round(np.mean(np.abs(error)), 3)),
    str(round(np.sqrt(np.mean(np.abs(error)*np.abs(error))), 3)),
))